<a href="https://colab.research.google.com/github/MirzaSaniya/Extracting-Data-from-Python-and-SQL-Saniya-Mirza/blob/main/Extracting_data_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reading data
* Pandas support ingesting data from a variety of sources
  * Flat file
      * read_csv()
      * read_table()
      * read_fwf()
      * read_msgpack()
  * Excel
      * read_excel()
  * JSON/HTML
      * read_json()
      * read_html()
  * SQL
      * read_sql()
      * read_sql_table()
      * read_sql_query()
  * Other
      * read_feather()
      * read_hdf()
      * read_sas()
      * read_feather()
      * read_pickle()
      * read_gbq
* Supplementary reading
  * https://pandas.pydata.org/pandas-docs/stable/api.html#input-output

#### Flat files
* Flat files refer to local or remote files that have tabular data in them
* Properties
  * Each line refers to a single row
  * Each entry in the line corresponds to a single column value
  * Entries are separate by a **delimiter**
* Allowed locations
  * local file system
  * url
    * http
    * ftp
    * s3
    

In [ ]:
!cat names.txt

Name	Profession
Sharat	Data scientist
Bill Gates	Philanthropist
Tom Brady	Athlete
Bill Nye	Presenter


In [ ]:
import pandas as pd
df = pd.read_table(
    'names.txt',  # path/url to the file
    sep='\t',      # specify separator between columns
    header=0)     # row containing column names
df

,Name,Profession
0,Sharat,Data scientist
1,Bill Gates,Philanthropist
2,Tom Brady,Athlete
3,Bill Nye,Presenter


In [ ]:
# We can override column names
df = pd.read_table(
    'names.txt',
    header=0,
    names=['Full name', 'Current profession'])
df

,Full name,Current profession
0,Sharat,Data scientist
1,Bill Gates,Philanthropist
2,Tom Brady,Athlete
3,Bill Nye,Presenter


* Important options for read_table()/read_csv()
  * true_values: list of values that mean True
  * false_values: list of values that mean False
  * na_values : list of values thatmean NA/NAN
  * usecols: list of column indices or names to read
  * parse_dates: boolean or list of intes or names of columns to consider for
      parsing dates
  * nrows: number of rows to read
  * compression: 'infer', 'gzip', 'bz2', 'zip', 'xz'
  * comment: comment indicator. Rest of the line is skipped
  * dtypes: Both read_table() and read_csv() infer the data type from the values available. Data types can be explicity specified as a dict of column name: numpy dtype

#### CSV files
* CSV (Comma separated values) are format of choice to exchange tabular data
    * Each line corresponds to a row
    * Each entry in the line corresponds to a column
    * Entries are separated by comma (,)
    * The first row contains the names of the columns
* Its a special case of tabular data with sep=','

In [ ]:
!cat names.csv

Name,Profession
Sharat,Data scientist
Bill Gates,Philanthropist
Tom Brady,Athlete
Bill Nye,Presenter


In [ ]:
df = pd.read_csv('names.csv')
df

,Name,Profession
0,Sharat,Data scientist
1,Bill Gates,Philanthropist
2,Tom Brady,Athlete
3,Bill Nye,Presenter


In [ ]:
# Remote location
df = pd.read_csv(
    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


#### Excel file
* Flat files are organized into 'row's and 'column's
* Excel files are organized
  * sheet
  * row
  * column
* A single sheet correspondns to one or more sets of tabular data
* Example
<table>
<td><img src="excel_file.png"></td>
<td><img src="excel_file_2.png"></td>
</table>

In [ ]:
profession_df = pd.read_excel(
    'profession_and_age.xlsx', # location or url to the file
    sheetname='Profession' #name of the sheet
)
profession_df

/Users/schikkerur/anaconda2/lib/python2.7/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


,First Name,Last Name,Profession
0,Sharat,Chikkerur,Data scientist
1,Tom,Brady,Athlete
2,Bill,Nye,TV Personality
3,Bill,Gates,Philanthropist


In [ ]:
age_df = pd.read_excel(
    'profession_and_age.xlsx', # location or url to the file
    sheetname='Age' #name of the sheet
)
age_df

,First Name,Last Name,Age
0,Sharat,Chikkerur,38
1,Tom,Brady,40
2,Bill,Nye,62
3,Barack,Obama,56


In [ ]:
profession_df.merge(age_df, on=['First Name', 'Last Name'], how='outer')

,First Name,Last Name,Profession,Age
0,Sharat,Chikkerur,Data scientist,38.0
1,Tom,Brady,Athlete,40.0
2,Bill,Nye,TV Personality,62.0
3,Bill,Gates,Philanthropist,NaN
4,Barack,Obama,NaN,56.0


#### Additional options
```python
pandas.read_excel(io, sheet_name=0, header=0, skiprows=None, skip_footer=0, index_col=None, names=None, usecols=None, parse_dates=False, date_parser=None, na_values=None, thousands=None, convert_float=True, converters=None, dtype=None, true_values=None, false_values=None, engine=None, squeeze=False, **kwds)
```
* **skiprows** : Rows to skip in the beginning
* **skip_footer**: Rows to skip in the end
* **names** : List of column names to use
* **header** : None if there is no header, otherwise row number to use
* **usecols** : Selectively include columns
* **na_values** : Additional string to recognize as NA

* Supplementary reading
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.htm

### Reading data from SQL
* SQL sources store data in relational form
  * Data is organized into databases
  * Each database consists of one or more tables
  * Each table consists of rows and columns
* Benefits of SQL
  * Tables can be logically linked using joins
  * Row and column subsets can be expressed through SQL queries
* Pandas
  * pandas.io.sql module provides a collection of wrappers to connect to SQL databases
  * Its abstracts the DB specific APIs (requires sqlalchemy to be installed)
  * Steps
    * Create a connection using sqlalchemy.create_connection()
    * Access data using table name or query
        * read_sql_table()
        * read_sql_query()
        * read_sql()

### Accessing  databases
* For this exercise, we will try to access a local SQL database using sqlite.
* Procedure to access remote database are similar.
* The local database was created in SQLite using the following
```sql
sqlite> .open profession_and_age
sqlite> create table profession(first_name varchar ,last_name varchar, profession varchar);
sqlite> create table age(first_name varchar, last_name varchar, age int);
sqlite> insert into profession values (("Sharat", "Chikkerur", "Data scientist"), ("Tom", "Brady", "Athlete"), ("Bill", "Nye", "TV Personality"), ("Bill", "Gates", "Philanthropist"));
Error: table profession has 3 columns but 4 values were supplied
sqlite> insert into profession values ("Sharat", "Chikkerur", "Data scientist"), ("Tom", "Brady", "Athlete"), ("Bill", "Nye", "TV Personality"), ("Bill", "Gates", "Philanthropist");
sqlite> insert into age values("Sharat","Chikkerur", 38), ("Tom", "Brady", 40), ("Bill", "Nye", 62), ("Barack", "Obama", 56);
```

In [ ]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///profession_and_age")

In [ ]:
pd.read_sql_table('profession', engine)

,first_name,last_name,profession
0,Sharat,Chikkerur,Data scientist
1,Tom,Brady,Athlete
2,Bill,Nye,TV Personality
3,Bill,Gates,Philanthropist


In [ ]:
pd.read_sql_table('age', engine)

,first_name,last_name,age
0,Sharat,Chikkerur,38
1,Tom,Brady,40
2,Bill,Nye,62
3,Barack,Obama,56


In [ ]:
pd.read_sql_query(
"""
select p.first_name, p.last_name, p.profession, a.age
from
    profession p
    left join age a
    on
        p.first_name = a.first_name and
        p.last_name = a.last_name
""", engine)

,first_name,last_name,profession,age
0,Sharat,Chikkerur,Data scientist,38.0
1,Tom,Brady,Athlete,40.0
2,Bill,Nye,TV Personality,62.0
3,Bill,Gates,Philanthropist,NaN


#### Scraping data from the web
* Data on the web might be
  * csv, excel, flat files available through a URL
  * structured data available through an API
  * unstructured data embedded as parts of a web page
* Flat files
  * read_csv(), read_excel(), read_table() support URL as location of files
* Structure data:
  * requests library allows us to read/write to web-apis
* Unstructured data
  * BeautifulSoup library allows us to parse html pages and extract information

#### Structured data
* RESTful APIs allow us to query data by placing http calls
* requests library allows us to access data available through http(s) protocol
* Example:
  * Coindesk provides a http api to get current bitcoin prices
    * https://www.coindesk.com/api/
  * We can use requests library to access current price
```python
import requests
URL = 'https://api.coindesk.com/v1/bpi/currentprice/USD.json'
response = requests.get(URL)
response.json()
```
* A list of open access APIs is curated at
https://github.com/toddmotto/public-apis

In [ ]:

import requests
URL = 'https://api.coindesk.com/v1/bpi/currentprice/USD.json'
response = requests.get(URL)
json = response.json()

# Here is the entire result
print(json)
print

# We can access specific fields through the json attribute
print("Current price")
print json.get('bpi').get('USD')

{u'bpi': {u'USD': {u'rate': u'11,556.8150', u'code': u'USD', u'description': u'United States Dollar', u'rate_float': 11556.815}}, u'disclaimer': u'This data was produced from the CoinDesk Bitcoin Price Index (USD). Non-USD currency data converted using hourly conversion rate from openexchangerates.org', u'time': {u'updated': u'Mar 5, 2018 12:11:00 UTC', u'updateduk': u'Mar 5, 2018 at 12:11 GMT', u'updatedISO': u'2018-03-05T12:11:00+00:00'}}

Current price
{u'rate': u'11,556.8150', u'code': u'USD', u'description': u'United States Dollar', u'rate_float': 11556.815}


#### Example
* Use metaweather API to get weather in london next two days
    * https://www.metaweather.com/api/

In [ ]:
# Get geocode for London
response =  requests.get(
    'https://www.metaweather.com/api/location/search/?query=london')
response.json()

[{u'latt_long': u'51.506321,-0.12714',
  u'location_type': u'City',
  u'title': u'London',
  u'woeid': 44418}]

In [ ]:
# Use geocode to access weather next two days
response = requests.get(
    'https://www.metaweather.com/api/location/44418')
json = response.json()
json.get('consolidated_weather')[0:2]

[{u'air_pressure': 992.76,
  u'applicable_date': u'2018-03-05',
  u'created': u'2018-03-05T11:33:02.773380Z',
  u'humidity': 83,
  u'id': 4799651341402112,
  u'max_temp': 9.956,
  u'min_temp': 4.814,
  u'predictability': 73,
  u'the_temp': 9.415,
  u'visibility': 8.860753201304382,
  u'weather_state_abbr': u's',
  u'weather_state_name': u'Showers',
  u'wind_direction': 161.3957510869405,
  u'wind_direction_compass': u'SSE',
  u'wind_speed': 7.338281480407221},
 {u'air_pressure': 990.275,
  u'applicable_date': u'2018-03-06',
  u'created': u'2018-03-05T11:33:03.173140Z',
  u'humidity': 81,
  u'id': 5979707833057280,
  u'max_temp': 8.936,
  u'min_temp': 3.168,
  u'predictability': 73,
  u'the_temp': 7.789999999999999,
  u'visibility': 9.500144158116598,
  u'weather_state_abbr': u's',
  u'weather_state_name': u'Showers',
  u'wind_direction': 226.18468831465609,
  u'wind_direction_compass': u'SW',
  u'wind_speed': 6.0253641439560965}]

#### Scraping data from the web
* When possible use APIs to access data
* In instances where APIs are not available, scraping can be used to parse the contents back to structured data
* Note:
  * Do not violate terms of service when scraping
  * Set a rate limit to scraping to avoid taking down website
* Process
  * Extract raw html
  * Use BeautifulSoup library to parse contents
  * Extract information from specific sections of the page

In [ ]:
page = open('html.html').read()
print(page)


<html>  
    <head>
    </head>
    <body>
        <h1> First title </h1>
        <h1> Second title </h1>
	<p class="content"> First paragraph </p>
	<p class="description"> Second paragraph </p>
    <body>
</html>



In [ ]:
from IPython.core.display import display, HTML
display(HTML(page))

In [ ]:
# Extracting data from the web page
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, 'html.parser')
print(soup.prettify())

<html>
 <head>
 </head>
 <body>
  <h1>
   First title
  </h1>
  <h1>
   Second title
  </h1>
  <p class="content">
   First paragraph
  </p>
  <p class="description">
   Second paragraph
  </p>
  <body>
  </body>
 </body>
</html>



In [ ]:
soup.find_all('h1')

[<h1> First title </h1>, <h1> Second title </h1>]

In [ ]:
#Extracting content of the heading
for heading in soup.find_all('h1'):
    print(heading.get_text())

 First title 
 Second title 


In [ ]:
# We can use attributes of the tag to extract specific tags
soup.find_all('p')

[<p class="content"> First paragraph </p>,
 <p class="description"> Second paragraph </p>]

In [ ]:
# We can use attributes of the tag to extract specific tags
soup.find_all('p', class_="content")

[<p class="content"> First paragraph </p>]